<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/vision3d/live3d_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/transpchan/Live3D-v2.git
%cd Live3D-v2
!pip install -r requirements.txt

In [2]:
# download weights
!mkdir weights
#!curl -O -J -L  https://github.com/transpchan/Live3D-v2/releases/download/checkpoints-2.1/checkpoints.zip
!curl -O -J -L  https://github.com/transpchan/Live3D-v2/releases/download/checkpoints-2.2/checkpoints.zip
!unzip checkpoints.zip -d ./weights/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  423M  100  423M    0     0  39.0M      0  0:00:10  0:00:10 --:--:-- 31.5M
curl: Saved to filename 'checkpoints.zip'
Archive:  checkpoints.zip
  inflating: ./weights/cinnnet.pth   
  inflating: ./weights/rgbadecodernet.pth  
  inflating: ./weights/udpadecodernet.pth  


In [3]:
character = 'double_ponytail' #@param ['double_ponytail', 'short_hair', 'self_defined']

In [ ]:
!rm *.zip.*
!rm -r character_sheet/
!rm -r poses/
!mkdir character_sheet/
if character == 'short_hair':
  !curl -O -J -L  https://github.com/transpchan/Live3D-v2/releases/download/samples/short_hair_images.zip
  !unzip -j  short_hair_images.zip -x '__MACOSX/*'  -d character_sheet/character/ 
elif character == 'double_ponytail':
  !curl -O -J -L  https://github.com/transpchan/Live3D-v2/releases/download/samples/double_ponytail_images.zip
  !unzip -j  double_ponytail_images.zip -x '__MACOSX/*' -d character_sheet/character/
else:
  print("Please upload your character sheets to /content/CoNR/character_sheet/ ")
if character == 'short_hair':
  !curl -O -J -L  https://github.com/transpchan/Live3D-v2/releases/download/samples/short_hair.zip
  !unzip -j  short_hair.zip -d poses/
elif character == 'double_ponytail':
  !curl -O -J -L  https://github.com/transpchan/Live3D-v2/releases/download/samples/double_ponytail.zip
  !unzip -j double_ponytail.zip -d poses/ 
else:
  print("Please upload your UDP sequences or poses images to /content/CoNR/poses/ .")

In [ ]:
#@title Show all character sheets
from IPython.display import Image,display
from pathlib import Path
path ='./character_sheet/'
imgs = []
for file in Path(path).rglob('*.[PpWw][NnEe][GgBb]*'):
          imgs.append(Image(filename=str(file), width=200))
          
print("Num of character sheets:", len(imgs))
display(*imgs)

In [ ]:
#@title (Optional) Run UDP Detector
#@markdown This additional demo will show the results by running the udp detector on the character sheet. If you want to run it on the pose sequence, you need to change the code in train.py
!pip install open3d
!mkdir results
!python3 train.py --mode=test \
--test_input_poses_images=./character_sheet/character/ \
--test_input_person_images=./character_sheet/ \
--test_output_dir=./results/ \
--test_checkpoint_dir=./weights/  \
--test_output_udp=True \
--test_output_video=False \
--test_pose_use_parser_udp=True

In [ ]:
#@title (Optional) Visualzie UDP detection results

if __name__ == "__main__":
    import numpy as np
    import open3d as o3d

    npz = np.load('./results/udp_0.npz', allow_pickle=True)
    print("img count:", npz["udp"].shape[0])
    a = np.moveaxis(npz["udp"][:, :, :, :], [2, 3], [0, 1]).reshape(-1, 4)
    img = np.moveaxis(npz["img"][:, :, :, :], [2, 3], [0, 1]).reshape(-1, 3)

    occulusion = (a[:, 3] > 0.90)

    xyz = a[occulusion, 0:3]
    rgb = img[occulusion, 0:3]
    print("points:", xyz.shape[0])
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz*[0.7, 0.25, 1])

    pcd.colors = o3d.utility.Vector3dVector(rgb)
    pcd.estimate_normals()
    pcd.orient_normals_consistent_tangent_plane(1)

    pcd2 = o3d.geometry.PointCloud()
    pcd2.points = o3d.utility.Vector3dVector([
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1],
    ])

    pcd2.paint_uniform_color([0.5, 0.5, 0.5])
    o3d.visualization.draw_plotly([pcd, pcd2])
    if False:
        print("Displaying pointcloud ...")
        o3d.visualization.draw([pcd])
    o3d.io.write_point_cloud("./pointcloud.ply", pcd)

In [8]:
#@title Run Video Generation
#@markdown For sample data, this process may take about 40 minutes. You can stop earlier to get an shorter result (by clicking stop on the left).
!mkdir results
!python3 train.py --mode=test \
--test_input_poses_images=./poses/ \
--test_input_person_images=./character_sheet/ \
--test_output_dir=./results/ \
--test_checkpoint_dir=./weights/ 

mkdir: cannot create directory ‘results’: File exists
world_size: 0
batch_size: 1
local_rank:  0
character sheet: ['character_sheet/character/未命名作品 1.png', 'character_sheet/character/未命名作品 2.png', 'character_sheet/character/未命名作品 3.png', 'character_sheet/character/未命名作品 4.png']
---
test images:  1459
---
Infer: 100%|█████████████████| 1459/1459 [1:48:37<00:00,  4.47s/it, data_time=4.3, train_time=0.542]


In [ ]:
#@title Convert video format for display
#@markdown `output.mp4` is the output with black background.  `output_adobe_premiere.mov` is the output with transparent background.
!ffmpeg -r 30 -y -i ./results/%d.png  -c:v qtrle output_adobe_premiere.mov 
!ffmpeg -r 30 -y -i ./results/%d.png  -c:v libx264 -strict -2 -pix_fmt yuv420p   output.mp4 

In [ ]:
#@title Play the generated video!
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 600):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{video_url}" type="video/mp4">
</video>""")
show_video('output.mp4')

In [ ]:
# Tips
# motion diffusion modelと組み合わせて動画が作れるかどうかを試す。realtime chatみたいなのができないかを試す。